In [11]:
from ultralytics import YOLO

In [12]:
!yolo task=detect mode=predict model=yolov8n.pt conf=0.25 source='https://media.roboflow.com/notebooks/examples/dog.jpeg' save=True

Ultralytics YOLOv8.2.95 🚀 Python-3.9.16 torch-2.4.1+cpu CPU (AMD Ryzen 5 4600H with Radeon Graphics)
YOLOv8n summary (fused): 168 layers, 3,151,904 parameters, 0 gradients, 8.7 GFLOPs

Found https://media.roboflow.com/notebooks/examples/dog.jpeg locally at dog.jpeg
image 1/1 d:\Github\Project_GovID\notebooks\dog.jpeg: 640x384 1 person, 1 car, 1 dog, 886.5ms
Speed: 300.3ms preprocess, 886.5ms inference, 443.1ms postprocess per image at shape (1, 3, 640, 384)
Results saved to d:\Github\Project_GovID\static\runs\detect\predict4
💡 Learn more at https://docs.ultralytics.com/modes/predict


In [13]:
# %pip install scikit-learn

In [14]:
from sklearn.model_selection import train_test_split

In [15]:
!yolo predict model=yolo11n-seg.pt source='https://youtu.be/LNwODJXcvt4' imgsz=320

Traceback (most recent call last):
  File "C:\Anaconda\envs\streamlit\lib\runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Anaconda\envs\streamlit\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "d:\Github\Project_GovID\.venv\Scripts\yolo.exe\__main__.py", line 7, in <module>
    sys.exit(entrypoint())
  File "D:\Github\Project_GovID\.venv\lib\site-packages\ultralytics\cfg\__init__.py", line 803, in entrypoint
    model = YOLO(model, task=task)
  File "D:\Github\Project_GovID\.venv\lib\site-packages\ultralytics\models\yolo\model.py", line 23, in __init__
    super().__init__(model=model, task=task, verbose=verbose)
  File "D:\Github\Project_GovID\.venv\lib\site-packages\ultralytics\engine\model.py", line 145, in __init__
    self._load(model, task=task)
  File "D:\Github\Project_GovID\.venv\lib\site-packages\ultralytics\engine\model.py", line 297, in _load
    self.model, self.ckpt = attempt_load_one_weight(we

In [16]:
IMAGES_PATH = "../static/aadhar/images"
LABELS_PATH = "../static/aadhar/labels"
NOTES_PATH = "../static/aadhar/notes.json"

In [18]:
import os
labels = os.listdir(LABELS_PATH)



# Split data
train, test = train_test_split(labels, test_size=0.15, shuffle=True)
valid, test = train_test_split(test, test_size=0.2)

print(f"train: {len(train)}; valid: {len(valid)}; test: {len(test)}")

train: 29; valid: 4; test: 2


In [19]:
prefix = "../static/aadhar"

In [20]:
os.makedirs(f"{prefix}/test/images")
os.makedirs(f"{prefix}/test/labels")
os.makedirs(f"{prefix}/train/images")
os.makedirs(f"{prefix}/train/labels")
os.makedirs(f"{prefix}/valid/images")
os.makedirs(f"{prefix}/valid/labels")

FileExistsError: [WinError 183] Cannot create a file when that file already exists: '../static/aadhar/test/images'

In [21]:
import shutil

In [ ]:
def move_files_to_dir(files, dirname):
    ct = 0
    for label_filename in files:
        image_filename = f"{label_filename[:-4]}.png"
        if os.path.exists(f"{IMAGES_PATH}/{image_filename}"):
            print(f"{image_filename} , -> {label_filename}")
            ct = ct +1
            print(f"{dirname}/images/{image_filename}")
                  
            shutil.copy(f"{IMAGES_PATH}/{image_filename}", f"{dirname}/images/{image_filename}")
            shutil.copy(f"{LABELS_PATH}/{label_filename}", f"{dirname}/labels/{label_filename}")
    print(ct)

# Move splits to folders
move_files_to_dir(train, "../static/aadhar/train")
move_files_to_dir(test, "../static/aadhar/test")
move_files_to_dir(valid, "../static/aadhar/valid")

In [22]:
import json

descr_darknet = json.load(open(NOTES_PATH))

train_path = "../static/aadhar/train/images"
test_path = "../static/aadhar/test/images"
valid_path = "../static/aadhar/valid/images"

nc = len(descr_darknet["categories"])
names = [category["name"] for category in descr_darknet["categories"]]

print(
    f"train: {train_path}\n"
    f"test: {test_path}\n"
    f"val: {valid_path}\n\n"
    f"nc: {nc}\n"
    f"names: {names}",
)

train: ../static/aadhar/train/images
test: ../static/aadhar/test/images
val: ../static/aadhar/valid/images

nc: 6
names: ['name_value', 'gender_value', 'dob_key', 'dob_value', 'adhar_number_value', 'name_key']


In [23]:
import yaml

with open("../data.yaml", "w") as file:
    yaml.dump({
        "train": train_path,
        "test": test_path,
        "val": valid_path,
        "nc": nc,
        "names": [f'{name}' for name in names]
    }, stream=file, default_flow_style=None)

In [ ]:

!yolo task=detect mode=train model=yolov8n.pt data="../data.yaml" epochs=100 imgsz=800 plots=True

In [1]:
from ultralytics import YOLO

In [2]:
model = YOLO("D:/Github/Project_GovID/models/adhar-processor-yolov8.pt")

In [3]:
# Run batched inference on a list of images
results = model(["D:/Github/Project_GovID/static/aadhar/test/images/2.png", "D:/Github/Project_GovID/static/aadhar/test/images/6.png"])  # return a list of Results objects

# Process results list
for result in results:
    boxes = result.boxes  # Boxes object for bounding box outputs
    masks = result.masks  # Masks object for segmentation masks outputs
    keypoints = result.keypoints  # Keypoints object for pose outputs
    probs = result.probs  # Probs object for classification outputs
    obb = result.obb  # Oriented boxes object for OBB outputs
    result.show()  # display to screen
    result.save(filename="result.jpg")  # save to disk


0: 800x800 1 adhar_number_value, 1238.0ms
1: 800x800 1 adhar_number_value, 1 name_key, 1238.0ms
Speed: 194.0ms preprocess, 1238.0ms inference, 97.5ms postprocess per image at shape (1, 3, 800, 800)
